In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
from eth_defi.uniswap_v2.pair import PairDetails, fetch_pair_details

In [3]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
sync_filt = Filter.create_filter(address=None, event_types=[Pair.events.Sync])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 1000

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=sync_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])
    pair: PairDetails = fetch_pair_details(web3, evt['address'])

    amt0 = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    amt1 = Conversion().convert_int256_bytes_to_int(arguments[1])  
    amt0_human = amt0/(10**pair.token0.decimals)
    amt1_human = amt1/(10**pair.token1.decimals)    

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt['event']
    event['details']['token0'] = pair.token0.address
    event['details']['token1'] = pair.token1.address
    event['details']['token0_symbol'] = pair.token0.symbol
    event['details']['token1_symbol'] = pair.token1.symbol 
    event['details']['token0_decimal'] = pair.token0.decimals
    event['details']['token1_decimal'] = pair.token1.decimals        
    event['details']['amount0'] = amt0
    event['details']['amount1'] = amt1
    event['details']['price'] = amt1_human/amt0_human
    
    events[k] = event
    if key not in processed_events:
        print(f"Sync at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Sync at block:60,390,150 tx:0x78288df7933ab0f89b7c7130e93110c694fce2d77065e2744aba4d3bba5f41cb
Sync at block:60,390,151 tx:0x6dc32a5b6c24e226f12015534a348343e645ebaa47add6f5d917814cf0e98c36
Sync at block:60,390,151 tx:0xa28ffe3a7f94c7d0d2347a91219c4c5fa57f14695b09d6394fdc54cb8a1c561c
Sync at block:60,390,151 tx:0xa28ffe3a7f94c7d0d2347a91219c4c5fa57f14695b09d6394fdc54cb8a1c561c
Sync at block:60,390,151 tx:0xa28ffe3a7f94c7d0d2347a91219c4c5fa57f14695b09d6394fdc54cb8a1c561c
Sync at block:60,390,151 tx:0xa28ffe3a7f94c7d0d2347a91219c4c5fa57f14695b09d6394fdc54cb8a1c561c
Sync at block:60,390,153 tx:0xb801300ecd40aa7095ac7c724c306c76cfabcbec22a72f0cc8e44b6b4a0f004e
Sync at block:60,390,155 tx:0xc656588867eab0e71027f7d04eb5ea467add2e00818323288c70043289d6499f
Sync at block:60,390,155 tx:0xc656588867eab0e71027f7d04eb5ea467add2e00818323288c70043289d6499f
Sync at block:60,390,155 tx:0xc656588867eab0e71027f7d04eb5ea467add2e00818323288c70043289d6499f
Sync at block:60,390,155 tx:0xc656588867eab0e71027

ContractLogicError: execution reverted

In [ ]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

In [9]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0x125845099f9db1e4c7ce42768afece4ae55f7e8a',
  'tx_hash': '0x78288df7933ab0f89b7c7130e93110c694fce2d77065e2744aba4d3bba5f41cb',
  'blk_num': 60390150,
  'timestamp': 1723229317,
  'details': {'web3_type': web3._utils.datatypes.Sync,
   'token0': '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270',
   'token1': '0x8FAAD147AD9E1cDCcc97eF6FA3762680c8bA2637',
   'token0_symbol': 'WMATIC',
   'token1_symbol': 'NEIRO',
   'token0_decimal': 18,
   'token1_decimal': 18,
   'amount0': 27220568319074974380978,
   'amount1': 271677190661881126378,
   'price': 0.009980584809153368}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0x882df4b0fb50a229c3b4124eb18c759911485bfb',
  'tx_hash': '0x6dc32a5b6c24e226f12015534a348343e645ebaa47add6f5d917814cf0e98c36',
  'blk_num': 60390151,
  'timestamp': 1723229319,
  'details': {'web3_type': w